# PySpark

## 설치하기
PySpark를 이용하기 위해선 다음과 같은 패키지가 설치되어 있어야 합니다.
* java (>= 8.0)
* Spark (>= 2.2.0)
* Python (>= 3.4.0)


**<스파크 설치>**  
아래 코드를 통해 버전을 확인해 보세요. 2.2.X 이상이면 됩니다.

In [4]:
import pyspark
pyspark.__version__

AttributeError: module 'pyspark' has no attribute '__version__'

## 실습

In [ ]:
from pyspark import SparkConf, SparkContext

# 선언
sc = SparkContext()
sc

In [ ]:
type(sc)

In [ ]:
# 실수로 SparkContext를 한 개 더 만들면 에러가 납니다.
new_sc = SparkContext()

In [ ]:
# SparkContext를 종료
sc.stop()

In [ ]:
# 어플리케이션의 이름과 Master의 URL을 설정
conf = SparkConf().setAppName('PySpark Basic').setMaster('local')
sc = SparkContext(conf=conf)
sc

## RDD Creation

### 1) 내부에서 만들어진 데이터 집합을 병렬화하는 방법: parallelize()함수

In [ ]:
rdd = sc.parallelize([1,2,3])
rdd

In [ ]:
rdd.take(3)

### 2) 외부의 파일을 로드하는 방법: .textFile() 함수 사용
